## 1. Setup StockX

In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))


from ib_insync import *
util.startLoop()


ib = IB()
port = 7496
port = 4002
ib.connect('127.0.0.1', port, clientId=15)
# sx  = stock.StockX(ib, 'TSLA')
# spy = stock.StockX(ib, 'SPY')

<IB connected to 127.0.0.1:4002 clientId=15>

# -------------------------------------

In [2]:

from strategies.preset_strats import TAPresets1D, TAPresets1H
ta_1D = TAPresets1D(ls='LONG', isSpy=True, lookBack=150)
ta_1D.s_1D

1D :: LONG :: 150


Score(name='1Ds_mean', normRange=(0, 100), ls='LONG', lookBack=150, columnStartsWith='', chartArgs=ChartArgItem(style={'dash': 'solid', 'color': 'magenta', 'width': 2}, chartType='line', row=4, nameCol=None, columns=None, mask=None, name=None), invertScoreIfShort=False, sigs=[RoomToMove(name='RTM_L_Res_1_Lower', normRange=(0, 5), ls='LONG', lookBack=150, chartArgs=ChartArgItem(style={'dash': 'solid', 'color': 'pink', 'width': 3}, chartType='line', row=3, nameCol=None, columns=None, mask=None, name=None), invertScoreIfShort=False, tgetCol='Res_1_Lower', atrCol='ATR_14', unlimitedVal=5), GappedWRBs(name='L_GapWRBs', normRange=(0, 100), ls='LONG', lookBack=150, chartArgs=ChartArgItem(style={'dash': 'solid', 'color': 'orange', 'width': 3}, chartType='line', row=3, nameCol=None, columns=None, mask=None, name=None), invertScoreIfShort=False, bswCol='BarSW'), GappedPivots(name='L_GPivs', normRange=(0, 3), ls='LONG', lookBack=150, chartArgs=ChartArgItem(style={'dash': 'solid', 'color': 'orange

In [3]:
import stock
from strategies.preset_strats import TAPresets1D, TAPresets1H


# symbols to check for and why they are chosen. Gaps not on this days but charts have good previous gaps.
# symbol = 'SATX', date = '2025-04-01', size = '1 day', reason = 'Gapps'
# symbol = 'ECX',  date = '2025-04-01', size = '1 day', reason = 'Big Gap down and gap up '
# symbol = 'ANIX', date = '2025-04-01', size = '1 day', reason = 'Gap over resistance'
# symbol = 'SLQT', date = '2025-04-01', size = '1 day', reason = 'Big gap up '
# symbol = 'PRGS', date = '2025-04-01', size = '1 day', reason = 'Big Gap down and gap up '
# symbol = 'CFFN', date = '2025-04-01', size = '1 day', reason = 'Big Gap down and gap up '
# symbol = 'SRAD', date = '2025-04-01', size = '1 day', reason = 'Big gap up '

# TODO - Fix gaps. GPivs and Gapped past Pivots
# TODO - Fix RS needs to be Score 0,100 .  add score to TA maybe 
# TODO - Not sure Gapped Bars is working correctly. GapWRBs

sx  = stock.StockX(ib, 'UVIX') 
spy = stock.StockX(ib, 'SPY')


def RUN_DAILY(self, ls:str='', spy:object=None, isMarket:bool=False, displayCharts:bool=False, printStats:bool=False, forceDownload:bool=False):
    print(f"------------ StockX::RUN_DAILY: {self.symbol} {self.ls}------------------------------------------------------------------------")

    self.set_ls(ls)
    if isMarket:
        self.setup_frame('1 day', 'ohlcv', duration="200 D", endDateTime='now', isDayFrame=True,      taPresets=TAPresets1D(ls=self.ls, isSpy=True, lookBack=100))
        self.setup_frame('1 hour', 'ohlcv', duration="15 D", endDateTime='now', isIntradayFrame=True, taPresets=TAPresets1H(ls=self.ls, isSpy=True, lookBack=100))
        self.frames['1 day'].run_ta()
        self.frames['1 hour'].run_ta()

    else:
        self.setup_frame('1 day', 'ohlcv', duration="200 D", endDateTime='now', isDayFrame=True,      taPresets=TAPresets1D(ls=self.ls, lookBack=100))
        self.setup_frame('1 hour', 'ohlcv', duration="15 D", endDateTime='now', isIntradayFrame=True, taPresets=TAPresets1H(ls=self.ls, lookBack=100, volChgPctThreshold=50))
        self.import_all_market_data(spy)
        self.frames['1 day'].run_ta()
        self.frames['1 hour'].run_ta()

 

ls= 'LONG'
RUN_DAILY(spy, ls=ls, isMarket=True)
RUN_DAILY(sx, ls=ls, spy=spy, isMarket=False)

------------ StockX::RUN_DAILY: SPY ------------------------------------------------------------------------
1D :: LONG :: 150
SPY 1 day 200 D :: 2024-06-25 00:00:00 to 2025-04-09 00:00:00 rows: 199, missing: 0, Data stored as 1 day and resampled to 1 day
SPY 1 hour 15 D :: 2025-03-21 00:00:00 to 2025-04-09 18:00:00 rows: 226, missing: 0, Data stored as 1 hour and resampled to 1 hour
------------ StockX::RUN_DAILY: UVIX ------------------------------------------------------------------------
1D :: LONG :: 100
UVIX 1 day 200 D :: 2024-06-25 00:00:00 to 2025-04-09 00:00:00 rows: 199, missing: 0, Data stored as 1 day and resampled to 1 day
UVIX 1 hour 15 D :: 2025-03-21 00:00:00 to 2025-04-09 18:00:00 rows: 226, missing: 0, Data stored as 1 hour and resampled to 1 hour


In [4]:
# spy.get_frame_data('1 day').columns #  .taPresets.BarSW
# sx.get_frame_data('1 day').columns #  .taPresets.BarSW
spy.get_frame('1 day').taPresets.__dict__['ta_list'] # .RoomToMove

[ATR(column=None, name='ATR_14', names=['ATR_14'], chartArgs=ChartArgItem(style={}, chartType='', row=1, nameCol=None, columns=None, mask=None, name=None), hi_col='high', lo_col='low', span=14),
 HPLP(column=None, name=None, names=['HP_hi_10', 'LP_lo_10'], chartArgs=ChartArgItem(style=[{'color': 'green', 'size': 10}, {'color': 'red', 'size': 10}], chartType='points', row=1, nameCol=None, columns=None, mask=None, name=None), hi_col='high', lo_col='low', span=10),
 HPLP(column=None, name=None, names=['HP_hi_3', 'LP_lo_3'], chartArgs=ChartArgItem(style=[{'color': 'green', 'size': 5}, {'color': 'red', 'size': 5}], chartType='points', row=1, nameCol=None, columns=None, mask=None, name=None), hi_col='high', lo_col='low', span=3),
 SupResAllRows(column=None, name=None, names=['Res_1', 'Res_1_Upper', 'Res_1_Lower', 'Res_2', 'Res_2_Upper', 'Res_2_Lower', 'Sup_1', 'Sup_1_Upper', 'Sup_1_Lower', 'Sup_2', 'Sup_2_Upper', 'Sup_2_Lower'], chartArgs=ChartArgItem(style=[{'dash': 'solid', 'main_line_colo

In [5]:
sx.frames['1 day'].taPresets.get_ta_list()
sx.frames['1 hour'].taPresets.get_ta_list()


[ATR(column=None, name='ATR_14', names=['ATR_14'], chartArgs=ChartArgItem(style={}, chartType='', row=1, nameCol=None, columns=None, mask=None, name=None), hi_col='high', lo_col='low', span=14),
 HPLP(column=None, name=None, names=['HP_hi_10', 'LP_lo_10'], chartArgs=ChartArgItem(style=[{'color': 'green', 'size': 10}, {'color': 'red', 'size': 10}], chartType='points', row=1, nameCol=None, columns=None, mask=None, name=None), hi_col='high', lo_col='low', span=10),
 HPLP(column=None, name=None, names=['HP_hi_3', 'LP_lo_3'], chartArgs=ChartArgItem(style=[{'color': 'green', 'size': 5}, {'color': 'red', 'size': 5}], chartType='points', row=1, nameCol=None, columns=None, mask=None, name=None), hi_col='high', lo_col='low', span=3),
 SupResAllRows(column=None, name=None, names=['Res_1', 'Res_1_Upper', 'Res_1_Lower', 'Res_2', 'Res_2_Upper', 'Res_2_Lower', 'Sup_1', 'Sup_1_Upper', 'Sup_1_Lower', 'Sup_2', 'Sup_2_Upper', 'Sup_2_Lower'], chartArgs=ChartArgItem(style=[{'dash': 'solid', 'main_line_colo

In [6]:

# s_1D_sigs_names = [s_1D_sigs[i].name for i in range(len(s_1D_sigs))]
# s_1D_sigs_names

# s_1D_vals = spy.get_frame_data('1 day')[s_1D_sigs_names]
# s_1D_vals.iloc[-1]
spy.get_frame('1 day').taPresets.__dict__
spy.get_frame('1 day').taPresets.s_1D

Score(name='1Ds_mean', normRange=(0, 100), ls='LONG', lookBack=150, columnStartsWith='', chartArgs=ChartArgItem(style={'dash': 'solid', 'color': 'magenta', 'width': 2}, chartType='line', row=4, nameCol=None, columns=None, mask=None, name=None), invertScoreIfShort=False, sigs=[RoomToMove(name='RTM_L_Res_1_Lower', normRange=(0, 5), ls='LONG', lookBack=150, chartArgs=ChartArgItem(style={'dash': 'solid', 'color': 'pink', 'width': 3}, chartType='line', row=3, nameCol=None, columns=None, mask=None, name=None), invertScoreIfShort=False, tgetCol='Res_1_Lower', atrCol='ATR_14', unlimitedVal=5), GappedWRBs(name='L_GapWRBs', normRange=(0, 100), ls='LONG', lookBack=150, chartArgs=ChartArgItem(style={'dash': 'solid', 'color': 'orange', 'width': 3}, chartType='line', row=3, nameCol=None, columns=None, mask=None, name=None), invertScoreIfShort=False, bswCol='BarSW'), GappedPivots(name='L_GPivs', normRange=(0, 3), ls='LONG', lookBack=150, chartArgs=ChartArgItem(style={'dash': 'solid', 'color': 'orange

In [7]:
display(spy.get_frame_data('1 hour', colsContains='1H').tail(5))
display(spy.get_frame_data('1 day').tail(5))

,1H_mean,1H_mean_>_50,1Hv_mean,1Hv_mean_>=_100,1Hf_mean,1Hf_mean_>_50
date,,,,,,
2025-04-09 14:00:00,0.0,0.0,0,0,0.0,0.0
2025-04-09 15:00:00,0.0,0.0,0,0,0.0,0.0
2025-04-09 16:00:00,0.0,0.0,0,0,0.0,0.0
2025-04-09 17:00:00,0.0,0.0,0,0,0.0,0.0
2025-04-09 18:00:00,26.19,0.0,0,0,26.19,0.0


,open,high,low,close,volume,average,barCount,ATR_14,HP_hi_10,LP_lo_10,...,L_GPivs,L_GPP,vL_L_GapWRBs_>_50,vL_L_GPivs_>_1,vL_L_GPP_>_80,vL_RTM_L_Res_1_Lower_>_1,1Ds_mean,1Ds_mean_>=_50,1Dv_mean,1Dv_mean_>=_100
date,,,,,,,,,,,,,,,,,,,,,
2025-04-04,523.73,525.87,505.52,509.97,1003812.0,515.352,569526.0,9.387143,NaN,NaN,...,0.0,0.0,0,0,0,0,0.16,0.0,0.0,0
2025-04-07,489.19,523.17,481.80,509.18,1464317.0,500.698,904144.0,11.916429,NaN,481.8,...,0.0,0.0,0,0,0,100,0.46,0.0,25.0,0
2025-04-08,521.87,524.98,507.58,510.40,494296.0,517.861,281011.0,12.493571,NaN,NaN,...,0.0,0.0,0,0,0,100,11.62,0.0,25.0,0
2025-04-09,493.44,536.50,493.05,529.00,887997.0,510.735,510990.0,15.027857,NaN,NaN,...,0.0,0.0,0,0,0,100,3.47,0.0,25.0,0
2025-04-10,NaN,NaN,NaN,540.02,1276.0,NaN,NaN,15.656154,NaN,NaN,...,0.0,0.0,0,0,0,100,0.77,0.0,25.0,0


In [8]:
spy.get_frame('1 day').plot()
spy.get_frame('1 hour').plot()
sx.get_frame('1 day').plot()
sx.get_frame('1 hour').plot()

# ----------------------

In [9]:
spy.get_frame_data('1 day', colsContains='sv')



""
date
2024-06-25
2024-06-26
2024-06-27
2024-06-28
2024-07-01
...
2025-04-04
2025-04-07
2025-04-08


In [10]:
sx.get_frame_data('1 day', colsContains='Score')

""
date
2024-06-25
2024-06-26
2024-06-27
2024-06-28
2024-07-01
...
2025-04-04
2025-04-07
2025-04-08


In [11]:
# spy.RUN_DAILY(isMarket=True)

# sx.set_ls('SHORT')
# sx.RUN_DAILY(spy)

In [12]:
sx.get_frame_data('1 hour', colsContains='Score')

""
date
2025-03-21 00:00:00
2025-03-21 08:00:00
2025-03-21 09:00:00
2025-03-21 10:00:00
2025-03-21 11:00:00
...
2025-04-09 14:00:00
2025-04-09 15:00:00
2025-04-09 16:00:00


In [13]:
sx.set_daily_stats(printStats=True)

Error 430, reqId 6: The fundamentals data for the security specified is not available.failed to fetch, contract: Stock(symbol='UVIX', exchange='SMART', currency='USD')


StockX UVIX::set_daily_stats: Fundamentals are not found.


KeyError: ''

In [ ]:
sx.get_frame_data('1 day').tail(10)

In [ ]:
sx.get_frame('1 day').plot()

In [ ]:
sx.reqLiveBars() # waits 3 seconds for the data to come in

In [ ]:

data = sx.updateLiveOHLCV(['1 hour'], spy)
display(data)
sx.get_frame('1 hour').plot()

In [ ]:


# ! next do the stock X day data for 5 mins and 2  mins 

In [ ]:
sx.get_live_price()

## 2. Fundamentals

In [ ]:
sx.RUN_FUNDAMENTALS(allowedETFs=['XLY', 'XLK', 'XLC'], maxDaysOld=10)

## 3. Setup

In [ ]:
sx.RUN_SETUP(spy, dataType='ohlcv', forceDownload=False, endDate='2025-02-20')

## 4. PreMarket

In [ ]:
sx.RUN_PRE_MARKET('LONG', timeToRun='09:00', minPreMarketScore=50, displayCharts=True, printStats=True) # runs daily and premarket volume analysis

In [ ]:
sx.display_columns('1 day')
sx.display_columns('1 day', contains='SPY')

## 5. Intraday

In [ ]:
sx.RUN_INTRADAY(updateEverySeconds=60, myTradingTimes=[('9:35', '15:00')], maxTrades=5, displayCharts=True, logTrades=True)

In [ ]:
import pandas as pd
import strategies.preset_strats as ps
import strategies.signals as sig
import strategies.ta as ta
from typing import Dict, Any, List
from chart.chart import ChartArgs

sx.import_all_market_data(spy)
sx.set_ls('LONG')
sx.setup_TA_1D(lookBack=25, atrSpan=14, sigRow=3, validationRow=4)

if sx.day_score_passed(minScore=50):
    sx.setup_TA_intraday(lookBack=200, atrSpan=50, sigRow=3, validationRow=4)
    
    if sx.pre_market_passed(minScore=50):
        sx.RUN(updateEvery=30, myTradingTimes=[('9:35', '15:00')], maxTrades=5, displayCharts=False, logTrades=True)




spy.frames['1 day'].plot()
sx.frames['1 day'].plot()
sx.frames['1 hour'].plot()
sx.frames['5 mins'].plot()

In [ ]:
sx.display_columns('5 mins', contains='PB')

In [ ]:
dfx = sx.display_columns('5 mins', contains='PB')
dfx.loc['2025-02-20 09:35:00':].head(50)

 # Stratgey

In [ ]:
ib.disconnect()

In [ ]:
sx.get_frame('1 hour').taPresets.get_ta_list()
# prests.s_1H.passed
#  max rows
import pandas as pd
pd.set_option('display.max_rows', 1000)
sx.get_frame_data('1 hour', colsContains='1H').tail(30)

,1H_mean,1H_mean_>_50,1Hv_mean,1Hv_mean_>=_100,1Hf_mean,1Hf_mean_>_50
date,,,,,,
2025-04-04 21:00:00,16.13,0.0,0,0,16.13,0.0
2025-04-04 22:00:00,15.84,0.0,0,0,15.84,0.0
2025-04-04 23:00:00,15.3,0.0,0,0,15.3,0.0
2025-04-05 00:00:00,0.0,0.0,0,0,0.0,0.0
2025-04-07 00:00:00,100.0,1.0,100,100,100.0,1.0
2025-04-07 09:00:00,100.0,1.0,100,100,100.0,1.0
2025-04-07 10:00:00,100.0,1.0,100,100,100.0,1.0
2025-04-07 11:00:00,100.0,1.0,100,100,100.0,1.0
2025-04-07 12:00:00,100.0,1.0,100,100,100.0,1.0


In [ ]:

import pandas as pd
import numpy as np
from dataclasses import dataclass, field
from strategies.signals import Score, MultiSignals, Strategy
from strategies.preset_strats import TAPresets1D, TAPresets1H, ChartArgs
from typing import Dict, Any, List

@dataclass
class Strategy(MultiSignals):
    name: str
    chartArgs: List[ChartArgs] = field(default_factory=list)


    def __post_init__(self):
        self.names = []
        self.names_active = []
        self.names_base = [self.name, f"{self.name}_pctComplete", f"{self.name}_passed", f"{self.name}_meanScore"]
        self.names += self.names_base
        self.names_active += self.names_base
        self.steps = []
        self.rows = [] #  list of dicts for each row
        self.pctComplete = 0
        self.meanScoreOfSteps = 0
        self.current_step = 1
        self.passed = False
        self.scoreList = []
        self.chartArgs = []
        self.count_steps_passed = 0

        

    
    def add_chart_args(self, meanScoreArgs:ChartArgs, pctCompleteArgs:ChartArgs, scoreArgs:ChartArgs, failArgs:ChartArgs, scoreSubItemArgs:ChartArgs):
        meanScoreArgs.name = f"{self.name}_meanScore"
        meanScoreArgs.columns = [meanScoreArgs.name]
        pctCompleteArgs.name = f"{self.name}_pctComplete"
        pctCompleteArgs.columns = [pctCompleteArgs.name]
        scoreArgs.name = f"{self.name}_score"
        scoreArgs.columns = []
        failArgs.name = f"{self.name}_fail"
        failArgs.columns = []
        scoreSubItemArgs.name = f"{self.name}_subItem"
        scoreSubItemArgs.columns = []
        self.chartArgs = [meanScoreArgs, pctCompleteArgs, scoreArgs, failArgs, scoreSubItemArgs]
        return self
    
    def add_step(self, scoreObj:Score, failObj:Score, ifFailStartFromStep:int):
        self.steps.append({'scoreObj': scoreObj, 'failObj': failObj, 'ifFailStartFromStep': ifFailStartFromStep})
        step = len(self.steps)
        self.chartArgs[2].columns += [self.get_name(step, scoreObj.name)]
        self.chartArgs[3].columns += [self.get_name(step, failObj.name)]
        self.chartArgs[4].columns += [self.get_name(step, sig.name, parentName=scoreObj.name) for sig in scoreObj.sigs]
        return self
    
    def get_name(self, step:int, objName, parentName:str=None):
        name = f"{self.name}{step}__{objName}"
        if parentName is not None:
            name = f"{self.name}{step}__{parentName}_({objName})"
        if name not in self.names:  # Avoid duplicates
            self.names.append(name)
        return name
    
    def add_row_item(self, df, obj, parentName:str=None):
        step = self.current_step
        name = self.get_name(step, obj.name, parentName=parentName)
        self.names_active.append(name)
        self.rows[-1][name] = df[obj.name].iat[-1] 
        return self
    
    def add_row_metrics(self):
        self.rows[-1][f"{self.name}_passed"] = self.passed
        self.rows[-1][f"{self.name}_pctComplete"] = self.pctComplete
        self.rows[-1][f"{self.name}_meanScore"] = self.meanScoreOfSteps
        return self
    
    def update_mean_score(self, df, scoreObj):
        score = df[scoreObj.name].iloc[-1]
        self.scoreList += [score]
        self.meanScoreOfSteps = sum(self.scoreList) / len(self.scoreList)

    def update_pct_complete(self):
        self.pctComplete = 0 if self.count_steps_passed == 0 else (self.count_steps_passed / len(self.steps)) * 100

    def reset(self, startFromStep:int=1):
        self.current_step = startFromStep
        self.count_steps_passed = startFromStep - 1
        self.scoreList = self.scoreList[:startFromStep-1] # keep the scores from the failed step
        self.meanScoreOfSteps = 0 if len(self.scoreList) == 0 else sum(self.scoreList) / len(self.scoreList)
        self.update_pct_complete()
        self.passed = False
  


    def _compute_row(self, df: pd.DataFrame) -> pd.Series:
        if self.passed > 0:
            self.reset(1) # reset the strategy if it has passed

        self.rows += [{name:np.nan for name in self.names_base}]  

        for i, step in enumerate(self.steps):
            step_num = i + 1
            if step_num >= self.current_step:
                scoreObj = step['scoreObj']
                failObj = step['failObj']
                scorePassed = df[scoreObj.name_passed].iloc[-1]
                failed = df[failObj.name_passed].iloc[-1]

                if pd.isna(scorePassed):
                    break

                self.add_row_item(df, scoreObj)
                self.add_row_item(df, failObj)
                for sig in scoreObj.sigs:
                    self.add_row_item(df, sig, parentName=scoreObj.name)
                
                if failed > 0:
                    self.reset(step['ifFailStartFromStep'])
                    break
            
                if scorePassed > 0:
                    self.count_steps_passed += 1
                    self.update_mean_score(df, scoreObj)
                    
                    if self.current_step == len(self.steps):
                        self.passed = True
                    else:
                        self.current_step += 1

        self.update_pct_complete()
        self.add_row_metrics()
    
        # Create a Series with NaN values for all expected columns
        result = pd.Series(index=self.names_active, dtype='float64')
        
        # Fill in the values that we have
        for key, value in self.rows[-1].items():
            if key in result.index:
                result[key] = value


  
        return result

    def compute_signals(self, df: pd.DataFrame) -> pd.DataFrame:
        
        # Instead of pre-allocating, collect all results first
        all_results = []
        all_columns = set()
        
        for i in range(len(df)):
            row_result = self._compute_row(df.iloc[:i+1])
            
            # Convert to dict if it's not already one
            if isinstance(row_result, (np.ndarray, list)):
                # If your _compute_row returns arrays, you need a way to map them to column names
                # This is just an example - adjust based on your actual data structure
                row_dict = {f"col_{j}": val for j, val in enumerate(row_result)}
            else:
                row_dict = row_result
                
            # Track all column names we've seen
            all_columns.update(row_dict.keys())
            
            # Store the result with its index
            all_results.append((df.index[i], row_dict))
        
        # Now create a DataFrame with all possible columns
        results = pd.DataFrame(index=df.index, columns=list(all_columns))
        
        # Fill in the values
        for idx, row_dict in all_results:
            for col, val in row_dict.items():
                results.loc[idx, col] = val
                
        return results

#! TODO -- plot so each step can be viewed one ata a time.  Will maybe need to mask the chart df for this sepcial case 
ca = ChartArgs()

prests = sx.get_frame('1 hour').taPresets
strat = Strategy(name='SID').add_chart_args(ca.StratMeanScore, ca.StratPctComplete, ca.StratScores, ca.StratFails,  ca.StratSubItems)
strat.add_step(scoreObj=prests.s_1H, failObj=prests.s_1H_fail, ifFailStartFromStep=1)
strat.add_step(scoreObj=prests.sv_1H, failObj=prests.s_1H_fail, ifFailStartFromStep=1)
strat.add_step(scoreObj=prests.sv_1H, failObj=prests.s_1H_fail, ifFailStartFromStep=1)

strat.steps
results = strat.compute_signals(sx.get_frame_data('1 hour'))
print(f"Strategy: {strat.name}")
# max rows
pd.set_option('display.max_rows', 1000)
display(results.round(2).tail(100))

Strategy: SID


,SID_meanScore,SID1__1Hv_mean_(vL_VOL_TODC_100_>_50),SID1__1H_mean,SID1__1Hv_mean,SID_passed,SID1__1Hf_mean,SID,SID1__1H_mean_(VOL_TODC_100),SID_pctComplete
date,,,,,,,,,
2025-04-01 16:00:00,0,0,0.0,0,False,0.0,NaN,-56.034888,0.0
2025-04-01 17:00:00,0,0,0.0,0,False,0.0,NaN,-48.853673,0.0
2025-04-01 18:00:00,0,0,0.0,0,False,0.0,NaN,-46.948212,0.0
2025-04-01 19:00:00,0,0,0.0,0,False,0.0,NaN,-43.110678,0.0
2025-04-01 20:00:00,0,0,0.0,0,False,0.0,NaN,-37.613311,0.0
2025-04-01 21:00:00,0,0,0.0,0,False,0.0,NaN,-37.427247,0.0
2025-04-01 22:00:00,0,0,0.0,0,False,0.0,NaN,-37.358181,0.0
2025-04-01 23:00:00,0,0,0.0,0,False,0.0,NaN,-37.150848,0.0
2025-04-02 00:00:00,0,0,0.0,0,False,0.0,NaN,-99.755248,0.0


In [ ]:
import pandas as pd
import numpy as np
from dataclasses import dataclass, field
from typing import Dict, Any, List, Optional, Set, Union, TypeVar
from strategies.signals import Score, MultiSignals
from strategies.preset_strats import TAPresets1D, TAPresets1H, ChartArgs

T = TypeVar('T', bound='Strategy')

@dataclass
class Strategy(MultiSignals):
    """
    A multi-step strategy implementation that tracks progression through a sequence of
    scoring steps, handling success and failure conditions.
    
    Attributes:
        name: Unique identifier for the strategy
        chartArgs: List of chart configuration objects for visualization
    """
    name: str
    chartArgs: List[ChartArgs] = field(default_factory=list)

    def __post_init__(self) -> None:
        """Initialize strategy state after object creation."""
        # Create base column names used by the strategy
        self.names_base = [
            self.name, 
            f"{self.name}_pctComplete", 
            f"{self.name}_passed", 
            f"{self.name}_meanScore"
        ]
        
        # Track all column names (including dynamic ones added later)
        self.names: Set[str] = set(self.names_base)
        self.names_active: Set[str] = set(self.names_base)
        
        # Strategy execution state
        self.steps = []
        self.rows = []  # list of dicts for each row
        self.pctComplete = 0
        self.meanScoreOfSteps = 0
        self.current_step = 1
        self.passed = False
        self.scoreList = []
        self.chartArgs = []
        self.count_steps_passed = 0

    def add_chart_args(self, 
                     meanScoreArgs: ChartArgs, 
                     pctCompleteArgs: ChartArgs, 
                     scoreArgs: ChartArgs, 
                     failArgs: ChartArgs, 
                     scoreSubItemArgs: ChartArgs) -> T:
        """
        Configure chart visualization arguments for the strategy.
        
        Args:
            meanScoreArgs: Chart configuration for mean score display
            pctCompleteArgs: Chart configuration for percent complete display
            scoreArgs: Chart configuration for individual scores
            failArgs: Chart configuration for failure indicators
            scoreSubItemArgs: Chart configuration for sub-score components
            
        Returns:
            Self for method chaining
        """
        # Set up chart argument names and columns
        meanScoreArgs.name = f"{self.name}_meanScore"
        meanScoreArgs.columns = [meanScoreArgs.name]
        
        pctCompleteArgs.name = f"{self.name}_pctComplete"
        pctCompleteArgs.columns = [pctCompleteArgs.name]
        
        scoreArgs.name = f"{self.name}_score"
        scoreArgs.columns = []
        
        failArgs.name = f"{self.name}_fail"
        failArgs.columns = []
        
        scoreSubItemArgs.name = f"{self.name}_subItem"
        scoreSubItemArgs.columns = []
        
        self.chartArgs = [meanScoreArgs, pctCompleteArgs, scoreArgs, failArgs, scoreSubItemArgs]
        return self
    
    def add_step(self, scoreObj: Score, failObj: Score, ifFailStartFromStep: int) -> T:
        """
        Add a strategy execution step with associated scoring and failure handling.
        
        Args:
            scoreObj: Scoring object that determines success
            failObj: Scoring object that determines failure
            ifFailStartFromStep: Step to restart from in case of failure
            
        Returns:
            Self for method chaining
        """
        self.steps.append({
            'scoreObj': scoreObj, 
            'failObj': failObj, 
            'ifFailStartFromStep': ifFailStartFromStep
        })
        
        # Get the step number for naming
        step = len(self.steps)
        
        # Add column names to chart arguments
        self.chartArgs[2].columns.append(self.get_name(step, scoreObj.name))
        self.chartArgs[3].columns.append(self.get_name(step, failObj.name))
        
        # Add sub-signal column names
        for sig in scoreObj.sigs:
            sub_name = self.get_name(step, sig.name, parentName=scoreObj.name)
            self.chartArgs[4].columns.append(sub_name)
            
        return self
    
    def get_name(self, step: int, objName: str, parentName: Optional[str] = None) -> str:
        """
        Generate a unique name for a step component.
        
        Args:
            step: Step number
            objName: Name of the object
            parentName: Optional parent object name for hierarchical naming
            
        Returns:
            A unique identifier string
        """
        if parentName is not None:
            name = f"{self.name}{step}__{parentName}_({objName})"
        else:
            name = f"{self.name}{step}__{objName}"
            
        # Add to tracked names
        self.names.add(name)
        return name
    
    def add_row_item(self, df: pd.DataFrame, obj: Any, parentName: Optional[str] = None) -> T:
        """
        Add a single item to the current row.
        
        Args:
            df: DataFrame containing the data
            obj: Object with a 'name' attribute
            parentName: Optional parent object name for hierarchical naming
            
        Returns:
            Self for method chaining
        """
        step = self.current_step
        name = self.get_name(step, obj.name, parentName=parentName)
        self.names_active.add(name)
        self.rows[-1][name] = df[obj.name].iat[-1] 
        return self
    
    def add_row_metrics(self) -> T:
        """
        Add standard metrics to the current row.
        
        Returns:
            Self for method chaining
        """
        self.rows[-1][f"{self.name}_passed"] = self.passed
        self.rows[-1][f"{self.name}_pctComplete"] = self.pctComplete
        self.rows[-1][f"{self.name}_meanScore"] = self.meanScoreOfSteps
        return self
    
    def update_mean_score(self, df: pd.DataFrame, scoreObj: Score) -> None:
        """
        Update the running mean score based on the latest score.
        
        Args:
            df: DataFrame containing the score data
            scoreObj: Score object with scoring info
        """
        score = df[scoreObj.name].iloc[-1]
        self.scoreList.append(score)
        self.meanScoreOfSteps = np.mean(self.scoreList)

    def update_pct_complete(self) -> None:
        """Update the percentage completion of the strategy."""
        if self.count_steps_passed == 0:
            self.pctComplete = 0
        else:
            self.pctComplete = (self.count_steps_passed / len(self.steps)) * 100

    def reset(self, startFromStep: int = 1) -> None:
        """
        Reset the strategy state to a specific step.
        
        Args:
            startFromStep: Step number to reset to (default: 1)
        """
        self.current_step = startFromStep
        self.count_steps_passed = startFromStep - 1
        # Keep the scores from steps before the failure
        self.scoreList = self.scoreList[:startFromStep-1]
        
        # Recalculate mean score
        if len(self.scoreList) == 0:
            self.meanScoreOfSteps = 0
        else:
            self.meanScoreOfSteps = np.mean(self.scoreList)
            
        self.update_pct_complete()
        self.passed = False
  
    def _compute_row(self, df: pd.DataFrame) -> pd.Series:
        """
        Compute a single row of strategy signals.
        
        Args:
            df: DataFrame with data up to the current point
            
        Returns:
            Series containing computed signals for this row
        """
        # Reset strategy if it previously passed
        if self.passed:
            self.reset(1)

        # Initialize new row with base metrics as NaN
        self.rows.append({name: np.nan for name in self.names_base})

        # Process each step in sequence
        for i, step in enumerate(self.steps):
            step_num = i + 1
            
            # Only process current or future steps
            if step_num >= self.current_step:
                scoreObj = step['scoreObj']
                failObj = step['failObj']
                
                # Check if signals are available
                scorePassed = df[scoreObj.name_passed].iloc[-1]
                failed = df[failObj.name_passed].iloc[-1]

                # Break if score data is not available
                if pd.isna(scorePassed):
                    break

                # Add signals to the current row
                self.add_row_item(df, scoreObj)
                self.add_row_item(df, failObj)
                
                # Add sub-signals
                for sig in scoreObj.sigs:
                    self.add_row_item(df, sig, parentName=scoreObj.name)
                
                # Handle failure
                if failed > 0:
                    self.reset(step['ifFailStartFromStep'])
                    break
            
                # Handle success
                if scorePassed > 0:
                    self.count_steps_passed += 1
                    self.update_mean_score(df, scoreObj)
                    
                    # Check if all steps are complete
                    if self.current_step == len(self.steps):
                        self.passed = True
                    else:
                        self.current_step += 1

        # Update metrics and add to row
        self.update_pct_complete()
        self.add_row_metrics()
    
        # Create a Series with all expected columns initialized to NaN
        result = pd.Series(index=list(self.names_active), dtype='float64')
        
        # Fill in values from the current row
        for key, value in self.rows[-1].items():
            if key in result.index:
                result[key] = value

        return result

    def compute_signals(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Compute strategy signals for the entire DataFrame.
        
        Args:
            df: Input DataFrame with price/indicator data
            
        Returns:
            DataFrame with computed strategy signals
        """
        # Pre-allocate results with NaN
        results_data = []
        
        # Process each row sequentially
        for i in range(len(df)):
            # Compute row and store result with corresponding index
            row_result = self._compute_row(df.iloc[:i+1])
            results_data.append(row_result)
        
        # Combine all rows into a single DataFrame
        results = pd.DataFrame(results_data, index=df.index)
        
        return results

# Example usage:
ca = ChartArgs()
prests = sx.get_frame('1 hour').taPresets
strat = Strategy(name='SID').add_chart_args(ca.StratMeanScore, ca.StratPctComplete, ca.StratScores, ca.StratFails, ca.StratSubItems)
strat.add_step(scoreObj=prests.s_1H, failObj=prests.s_1H_fail, ifFailStartFromStep=1)
strat.add_step(scoreObj=prests.sv_1H, failObj=prests.s_1H_fail, ifFailStartFromStep=1)
strat.add_step(scoreObj=prests.sv_1H, failObj=prests.s_1H_fail, ifFailStartFromStep=1)
results = strat.compute_signals(sx.get_frame_data('1 hour'))
display(results.round(2).tail(100))

,SID_meanScore,SID_passed,SID_pctComplete,SID,SID1__1Hv_mean_(vL_VOL_TODC_100_>_50),SID1__1H_mean,SID1__1Hv_mean,SID1__1Hf_mean,SID1__1H_mean_(VOL_TODC_100)
date,,,,,,,,,
2025-04-01 16:00:00,0,False,0.0,NaN,0.0,0.00,0.0,0.00,-56.03
2025-04-01 17:00:00,0,False,0.0,NaN,0.0,0.00,0.0,0.00,-48.85
2025-04-01 18:00:00,0,False,0.0,NaN,0.0,0.00,0.0,0.00,-46.95
2025-04-01 19:00:00,0,False,0.0,NaN,0.0,0.00,0.0,0.00,-43.11
2025-04-01 20:00:00,0,False,0.0,NaN,0.0,0.00,0.0,0.00,-37.61
2025-04-01 21:00:00,0,False,0.0,NaN,0.0,0.00,0.0,0.00,-37.43
2025-04-01 22:00:00,0,False,0.0,NaN,0.0,0.00,0.0,0.00,-37.36
2025-04-01 23:00:00,0,False,0.0,NaN,0.0,0.00,0.0,0.00,-37.15
2025-04-02 00:00:00,0,False,0.0,NaN,0.0,0.00,0.0,0.00,-99.76
